# 🌍 Multilingual AI Translation System



## Problem Definition
Language barriers limit access to information, education, and global communication. Existing systems often support limited languages or require separate models for different language pairs.

## Objective
To design and implement a transformer-based multilingual translation system that:
- Automatically detects the source language
- Translates text into a user-selected target language
- Supports a wide range of global languages
- Provides an interactive interface for real-time usage

## Real-World Applications
- Global communication platforms
- Multilingual customer support
- Education and research
- Travel and tourism
- Digital inclusion for low-resource languages
 

## Technologies Used

- Programming Language: Python
- Deep Learning Framework: PyTorch
- NLP Model: Transformer (NLLB-200)
- Library: Hugging Face Transformers
- Language Detection: langdetect
- Interactive UI (Notebook): ipywidgets
- Deployment Framework: Streamlit (separate app)


INSTALL DEPENDENCIES

In [1]:
# Run only once if required
!pip install torch transformers langdetect ipywidgets sentencepiece


Defaulting to user installation because normal site-packages is not writeable


IMPORT LIBRARIES

In [10]:
import sys
!{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langdetect import detect
import ipywidgets as widgets
from IPython.display import display


## Data Understanding & Preparation

### Dataset Source
This project does not rely on a traditional static dataset.
Instead, it uses a pretrained multilingual transformer model (NLLB-200),
which has been trained on large-scale parallel multilingual text collected
from public sources by Meta AI.

### Data Handling
- User-provided text is treated as real-time input data
- No data is stored or persisted
- Text preprocessing is handled internally by the tokenizer


LOAD TRANSFORMER MODEL

In [3]:
model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model loaded successfully on:", device)


Model loaded successfully on: cpu



## Model / System Design

### AI Technique Used
- Deep Learning
- Natural Language Processing (NLP)
- Transformer-based Sequence-to-Sequence Model
- Large Language Model (LLM)

### Translation Pipeline
1. User enters text
2. System detects source language automatically
3. Text is tokenized using NLLB tokenizer
4. Transformer model generates translated tokens
5. Output is decoded into readable text

### Design Justification
- Transformers handle long-range dependencies effectively
- NLLB-200 supports 200+ languages using a single unified model
- No training required from scratch
- Scalable and industry-ready architecture


LANGUAGE MAPPING

In [4]:
LANG_NAME_TO_CODE = {
    "English": "eng_Latn",
    "Hindi": "hin_Deva",
    "Odia": "ory_Orya",
    "French": "fra_Latn",
    "German": "deu_Latn",
    "Spanish": "spa_Latn",
    "Bengali": "ben_Beng",
    "Tamil": "tam_Taml",
    "Telugu": "tel_Telu",
    "Marathi": "mar_Deva",
    "Urdu": "urd_Arab",
    "Italian": "ita_Latn",
    "Portuguese": "por_Latn",
    "Russian": "rus_Cyrl",
    "Japanese": "jpn_Jpan",
    "Korean": "kor_Hang",
    "Arabic": "arb_Arab",
    "Chinese": "zho_Hans"
}

DETECT_TO_MODEL_LANG = {
    "en": "eng_Latn",
    "hi": "hin_Deva",
    "bn": "ben_Beng",
    "ta": "tam_Taml",
    "te": "tel_Telu",
    "mr": "mar_Deva",
    "ur": "urd_Arab"
}


TRANSLATION FUNCTION

In [6]:
def translate_text(text, target_lang_code):
    try:
        detected_lang = detect(text)
    except:
        detected_lang = "en"

    if detected_lang not in DETECT_TO_MODEL_LANG:
        detected_lang = "en"

    tokenizer.src_lang = DETECT_TO_MODEL_LANG[detected_lang]

    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)

    outputs = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(target_lang_code),
        max_length=256
    )

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]


INTERACTIVE UI

In [7]:
text_input = widgets.Textarea(
    placeholder="Enter text in any language",
    description="Input:",
    layout=widgets.Layout(width="90%", height="100px")
)

language_dropdown = widgets.Dropdown(
    options=list(LANG_NAME_TO_CODE.keys()) + ["Other"],
    description="Target:"
)

other_language = widgets.Text(
    placeholder="Enter language name (e.g. Odia, French)",
    description="Other:"
)

translate_button = widgets.Button(
    description="Translate",
    button_style="success"
)

output_box = widgets.Textarea(
    description="Output:",
    layout=widgets.Layout(width="90%", height="100px")
)


In [ ]:
BUTTON LOGIC

In [8]:
def on_translate_clicked(b):
    if not text_input.value.strip():
        output_box.value = "Please enter input text."
        return

    if language_dropdown.value == "Other":
        lang_name = other_language.value.strip().capitalize()
    else:
        lang_name = language_dropdown.value

    if lang_name not in LANG_NAME_TO_CODE:
        output_box.value = "Selected language is not supported."
        return

    result = translate_text(
        text_input.value,
        LANG_NAME_TO_CODE[lang_name]
    )

    output_box.value = result

translate_button.on_click(on_translate_clicked)


In [ ]:
DISPLAY INTERFACE

In [9]:
display(
    text_input,
    language_dropdown,
    other_language,
    translate_button,
    output_box
)


Textarea(value='', description='Input:', layout=Layout(height='100px', width='90%'), placeholder='Enter text i…

Dropdown(description='Target:', options=('English', 'Hindi', 'Odia', 'French', 'German', 'Spanish', 'Bengali',…

Text(value='', description='Other:', placeholder='Enter language name (e.g. Odia, French)')

Button(button_style='success', description='Translate', style=ButtonStyle())

Textarea(value='', description='Output:', layout=Layout(height='100px', width='90%'))

## Evaluation & Analysis

### Metrics Used
This system is evaluated qualitatively based on:
- Semantic correctness
- Grammar and fluency
- Meaning preservation

### Observations
- High accuracy for high-resource languages
- Acceptable performance for low-resource languages
- Real-time inference supported on CPU


In [ ]:
## Ethical Considerations & Responsible AI

- Model performance may vary across languages
- Cultural nuances may not always be preserved
- No personal data is stored or logged
- System intended for educational and assistive purposes


In [ ]:
## Conclusion & Future Scope

### Conclusion
This project successfully demonstrates a transformer-based multilingual translation system with an interactive interface.
The system is suitable for academic evaluation and real-world deployment.

### Future Enhancements
- Speech-to-text and text-to-speech translation
- Document-level translation
- Model optimization for faster inference
- Mobile and API-based deployment
